# RFP: Maze Solvers

## Project Overview
You are invited to submit a proposal that answers the following question:

### What path will your elf take?

*Please submit your proposal by **2/11/25 at 11:59 PM**.*

## Required Proposal Components

### 1. Data Description
In the code cell below, use [Gymnasium](https://gymnasium.farama.org/) to set up a [Frozen Lake maze](https://gymnasium.farama.org/environments/toy_text/frozen_lake/) for your project. When you are done with the set up, describe the reward system you plan on using.

*Note, a level 5 maze is at least 10 x 10 cells large and contains at least five lake cells.*

In [3]:
import gymnasium as gym
import random
import pandas as pd
from gymnasium.envs.toy_text.frozen_lake import generate_random_map


In [4]:
# 0: "Left", 1: "Right", 2: "Up", 3: "Down"
# Just so I don't forget

In [5]:
maze=[
    "SFFHFFFFFH", 
    "FHFFFHHFFF", 
    "FFFHFHFFFH", 
    "FHFFFFFFFF", 
    "FFFFHFFHFH",
    "FHFHFHFHFH",
    "FFFFFFFFFF", 
    "FHFFFFFFFF",
    "HHFFFHFFFF",
    "FFFFFFFFGF"
]
env = gym.make('FrozenLake-v1', desc=maze)
initial_state = env.reset()
env.render()

C:\Users\hilo\anaconda3\Lib\site-packages\gymnasium\envs\toy_text\frozen_lake.py:329: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("FrozenLake-v1", render_mode="rgb_array")
  gym.logger.warn(


#### Reward System:
## Empty Space = -1 point
## Present = 100 points
## Hole = -100 points

### 2. Training Your Model
In the cell seen below, write the code you need to train a Q-Learning model. Display your final Q-table once you are done training your model.

*Note, level 5 work uses only the standard Python library and Pandas to train your Q-Learning model. A level 4 uses external libraries like Baseline3.*

In [8]:
# Learning rate - 0.5, discount - 0.5
# Bellman Equation: (1-alpha)q(s , a) + alpha(R + gamma(max(q(s' , a'))))
# Q Table Diagram

In [9]:
num_states = 100  
num_actions = 4 
grid_size = 10    

q = {state: [0] * num_actions for state in range(num_states)}

In [10]:
def updateQ(q, alpha, gamma, step, cell, reward):
    row = [q[3][cell], q[1][cell], q[0][cell], q[2][cell]]
    bell = (1-alpha)*(q[step][cell]) + alpha*(reward + (gamma*max(row)))
    q[step][cell] = bell

In [11]:
# Create my own reward system
cell_types = ['S', 'F', 'F', 'H', 'F', 'F', 'F', 'F', 'F', 'H', 'F', 'H', 'F', 'F', 'F', 'H', 'H', 'F', 'F', 'F', 'F', 'F', 'F', 'H', 'F', 'H', 'F', 'F', 'F', 'H', 'F', 'H', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'H', 'F', 'F', 'H', 'F', 'H', 'F', 'H', 'F', 'H', 'F', 'H', 'F', 'H', 'F', 'H', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'H', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'H', 'H', 'F', 'F', 'F', 'H', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'G', 'F']

def getReward(state):
    if cell_types[state] == "G":
        return 100
    elif cell_types[state] == "H":
        return -100
    else:
        return -1

In [12]:
# Q-learning parameters
alpha = 0.5     # Learning rate
gamma = 0.5     # Discount factor
epsilon = 0.7   # Exploration probability (increased for better learning)
max_steps = 250  # Max steps per episode
num_episodes = 5000000  # Reduce episodes for testing


In [13]:
def updateQ(q, alpha, gamma, state, action, reward, new_state):
    old_value = q[state][action]
    next_max = max(q[new_state]) if new_state in q else 0  # Ensure valid state
    new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
    q[state][action] = new_value

In [14]:
for episode in range(num_episodes):
    current_state = 0
    terminated = False
    steps = 0  

    while not terminated and steps < max_steps:
        # Epsilon-greedy action selection
        if random.uniform(0, 1) < epsilon:
            action = random.choice([0, 1, 2, 3])  # Explore
        else:
            action = q[current_state].index(max(q[current_state]))  # Exploit

        # Take action in the environment
        new_state, reward, terminated, truncated, info = env.step(action)
        reward = getReward(new_state)  # Assign reward

        # Update Q-table
        updateQ(q, alpha, gamma, current_state, action, reward, new_state)

        # Move to new state
        current_state = new_state
        steps += 1  

    env.reset()  

In [15]:
df = pd.DataFrame(q)
df = df.T
df

,0,1,2,3
0,-2.000019,-2.031605,-2.086798,-2.090677
1,-81.348685,-45.862657,-14.795016,-2.320658
2,-2.901851,-36.579019,-15.314041,-71.028260
3,0.000000,0.000000,0.000000,0.000000
4,-10.573591,-67.239824,-2.140696,-15.521024
...,...,...,...,...
95,-3.365752,-1.971135,-60.659376,-47.163516
96,-1.945558,-1.840947,-1.943591,-1.722307
97,-1.865166,-1.659298,-1.404486,-1.025288
98,0.000000,0.000000,0.000000,0.000000


### 3. Testing Your Model
In the cell seen below, write the code you need to test your Q-Learning model for **1000 episodes**. It is important to test your model for 1000 episodes so that we are all able to compare our results.

*Note, level 5 testing uses both a success rate and an average steps taken metric to evaluate your model. Level 4 uses one or the other.*

In [17]:
maze=["SFFHFFFFFH", "FHFFFHHFFF", "FFFHFHFFFH", "FHFFFFFFFF", "FFFFHFFHFH","FHFHFHFHFH","FFFFFFFFFF", "FHFFFFFFFF","HHFFFHFFFF","FFFFFFFFGF"]
env = gym.make('FrozenLake-v1', desc=maze)
initial_state = env.reset()
env.render()

C:\Users\hilo\anaconda3\Lib\site-packages\gymnasium\envs\toy_text\frozen_lake.py:329: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("FrozenLake-v1", render_mode="rgb_array")
  gym.logger.warn(


In [18]:
def takeAction(action):
    new_state, reward, terminated, truncated, info = env.step(action)
    return new_state, terminated

In [19]:
success_rate = 0

for i in range(1000):
    current_state = 0  # Reset to start state
    terminated = False
    steps = 0  

    while not terminated and steps < max_steps:
        # Print Q-values to debug
        print(f"State: {current_state}, Q-values: {q[current_state]}")

        # Choose best action
        action = q[current_state].index(max(q[current_state]))

        # Take action
        new_state, terminated = takeAction(action)

        # Ensure valid state transition
        if new_state is not None:
            current_state = new_state
        else:
            print("Invalid state, terminating episode.")
            terminated = True

        steps += 1  

    env.reset()  

    # Check if agent reached the goal
    if cell_types[current_state] == "G":
        success_rate += 1

# Print success rate
print(f"Success Rate: {(success_rate / 1000) * 100:.2f}%")

State: 0, Q-values: [-2.0000186746253457, -2.031605169595858, -2.0867983513340738, -2.090676839922308]
State: 0, Q-values: [-2.0000186746253457, -2.031605169595858, -2.0867983513340738, -2.090676839922308]
State: 0, Q-values: [-2.0000186746253457, -2.031605169595858, -2.0867983513340738, -2.090676839922308]
State: 0, Q-values: [-2.0000186746253457, -2.031605169595858, -2.0867983513340738, -2.090676839922308]
State: 0, Q-values: [-2.0000186746253457, -2.031605169595858, -2.0867983513340738, -2.090676839922308]
State: 0, Q-values: [-2.0000186746253457, -2.031605169595858, -2.0867983513340738, -2.090676839922308]
State: 0, Q-values: [-2.0000186746253457, -2.031605169595858, -2.0867983513340738, -2.090676839922308]
State: 10, Q-values: [-2.000072333021759, -19.48041074154771, -63.31241456772248, -14.644958887300293]
State: 10, Q-values: [-2.000072333021759, -19.48041074154771, -63.31241456772248, -14.644958887300293]
State: 0, Q-values: [-2.0000186746253457, -2.031605169595858, -2.08679835

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



State: 0, Q-values: [-2.0000186746253457, -2.031605169595858, -2.0867983513340738, -2.090676839922308]
State: 10, Q-values: [-2.000072333021759, -19.48041074154771, -63.31241456772248, -14.644958887300293]
State: 0, Q-values: [-2.0000186746253457, -2.031605169595858, -2.0867983513340738, -2.090676839922308]
State: 0, Q-values: [-2.0000186746253457, -2.031605169595858, -2.0867983513340738, -2.090676839922308]
State: 0, Q-values: [-2.0000186746253457, -2.031605169595858, -2.0867983513340738, -2.090676839922308]
State: 10, Q-values: [-2.000072333021759, -19.48041074154771, -63.31241456772248, -14.644958887300293]
State: 20, Q-values: [-2.0040871706138317, -2.0152029570419714, -4.2506229087292695, -2.8260905197369435]
State: 20, Q-values: [-2.0040871706138317, -2.0152029570419714, -4.2506229087292695, -2.8260905197369435]
State: 30, Q-values: [-2.056709730752691, -42.020729757115504, -2.308946977492269, -6.816319861484141]
State: 40, Q-values: [-2.171248489960525, -7.27331121733325, -6.741

In [20]:
action = q[current_state].index(max(q[current_state]))
current_state, terminated = takeAction(action)

In [21]:
max_q = df.iloc[current_state].max()
column_name = df.columns[(df == max_q).iloc[current_state]].tolist()
current_state, terminated = takeAction(int(column_name[0]))

In [22]:
env.close()

### 4. Final Answer
In the first cell below, describe the path your elf takes to get to the gift. *Note, a level 5 answer includes a gif of the path your elf takes in order to reach the gift.*

In the second cell seen below, describe how well your Q-Learning model performed. Make sure that you explicitly name the **learning rate**, **the discount factor**, and the **reward system** that you used when training your final model. *Note, a level 5 description describes the model's performance using two types of quantitative evidence.*

![example image](https://gymnasium.farama.org/_images/frozen_lake.gif)

#### Describe the path your elf takes here.

#### Describe how well your Q-Learning model performed here.